### Yielding from Generators

In [1]:
import math

In [6]:
# 1) create an iterator class
class FactIter:
    def __init__(self, n):
        self.n = n
        self.i = 0

    def __iter__(self):
        return self
    
    def __next__(self):
        if self.i >= self.n:
            raise StopIteration
        else:
            result = math.factorial(self.i)
            self.i += 1
            return result

In [7]:
fact_iter = FactIter(5)
list(fact_iter)

[1, 1, 2, 6, 24]

In [8]:
list(fact_iter)

[]

In [9]:
# 2) callable and sentinel value
def fact():
    i = 0
    def inner():
        nonlocal i
        result = math.factorial(i)
        i += 1
        return result
    return inner

In [10]:
f = fact()
f

<function __main__.fact.<locals>.inner()>

In [12]:
f(), f(), f(), f(), f()  # there is no upper limit

(120, 720, 5040, 40320, 362880)

In [19]:
fact_iter = iter(fact(), math.factorial(5))
list(fact_iter)

[1, 1, 2, 6, 24]

In [20]:
# (generator) factory fn
def my_func():
    print('line 1')
    yield 'Flying'
    print('line 2')
    yield 'Circus'

In [21]:
type(my_func)

function

In [22]:
f = my_func()

In [23]:
type(f)

generator

In [26]:
'__iter__' in dir(f), '__next__' in dir(f), iter(f) is f

(True, True, True)

In [30]:
f = my_func()
f.__next__()  # goes to first yield and executes everything in between

line 1


'Flying'

In [31]:
result = next(f)
result  # yield value is stored in var

line 2


'Circus'

In [32]:
def silly():
    yield 'the'
    yield 'ministry'
    yield 'of'
    yield 'silly'
    if True:
        return 'Sorry, all done'
    yield 'walks'

In [35]:
gen = silly()
next(gen), next(gen), next(gen), next(gen)

('the', 'ministry', 'of', 'silly')

In [36]:
next(gen)  # a return statement will exhaust a generator

StopIteration: Sorry, all done

In [39]:
# rewrite factorial iterator as a generator
def fact(n):
    for i in range(n):
        yield math.factorial(i)

In [41]:
gen = fact(5)
list(gen)

[1, 1, 2, 6, 24]

In [42]:
def squares(n):
    for i in range(n):
        yield i ** 2
list(squares(7))

[0, 1, 4, 9, 16, 25, 36]

### Fibonacci Sequence with generators

In [44]:
def fib_recursive(n):
    if n <= 1:
        return 1
    else:
        return fib_recursive(n-1) + fib_recursive(n-2)
[fib_recursive(i) for i in range(7)]

[1, 1, 2, 3, 5, 8, 13]

In [46]:
from timeit import timeit
timeit('fib_recursive(10)', globals=globals(), number=10)

0.0001604999999926804

In [48]:
timeit('fib_recursive(28)', globals=globals(), number=10)

1.9395979999999327

In [50]:
timeit('fib_recursive(29)', globals=globals(), number=10)  # the recursive strategy breaks down as n increases

2.9076109999998607

In [51]:
# let's use memoization to cache results

In [52]:
from functools import lru_cache

@lru_cache()
def fib_recursive(n):
    if n <= 1:
        return 1
    else:
        return fib_recursive(n-1) + fib_recursive(n-2)

In [54]:
timeit('fib_recursive(10)', globals=globals(), number=10), timeit('fib_recursive(28)', globals=globals(), number=10), timeit('fib_recursive(29)', globals=globals(), number=10)  # much faster!
# still a limit to the depth of recursion allowable by python

(3.600000127335079e-06, 3.099999958067201e-06, 2.7000000955013093e-06)

In [56]:
timeit('fib_recursive(2000)', globals=globals(), number=10)  # max recursion ):

RecursionError: maximum recursion depth exceeded in comparison

In [57]:
def fib(n):
    fib_0 = 1
    fib_1 = 1
    for i in range(n-1):
        fib_0, fib_1 = fib_1, fib_0 + fib_1
    return fib_1

[fib(i) for i in range(7)]

[1, 1, 2, 3, 5, 8, 13]

In [58]:
timeit('fib(5000)', globals=globals(), number=10)  # very fast and no recursion

0.006626599999890459

In [59]:
# create an iterator
class FibIter:
    def __init__(self, n):
        self.n = n
        self.i = 0

    def __iter__(self):
        return self
    
    def __next__(self):
        if self.i >= self.n:
            raise StopIteration
        else:
            result = fib(self.i)
            self.i += 1
            return result

In [60]:
fib_iter = FibIter(7)
list(fib_iter)

[1, 1, 2, 3, 5, 8, 13]

In [61]:
timeit('list(FibIter(5000))', globals=globals(), number=10)  # more abstracted, less efficient

16.624016299999994

In [62]:
def fib(n):
    fib_0 = 1
    fib_1 = 1
    for i in range(n-1):
        fib_0, fib_1 = fib_1, fib_0 + fib_1
        yield fib_1

In [63]:
gen = fib(7)
list(gen)

[2, 3, 5, 8, 13, 21]

In [71]:
def fib_gen(n):
    fib_0 = 1
    yield fib_0
    fib_1 = 1
    yield fib_1
    for _ in range(n-2):
        fib_0, fib_1 = fib_1, fib_0 + fib_1
        yield fib_1

In [72]:
gen = fib_gen(8)
list(gen)

[1, 1, 2, 3, 5, 8, 13, 21]

In [73]:
def fib_standard(n):
    fib_0 = 1
    fib_1 = 1
    for i in range(n-1):
        fib_0, fib_1 = fib_1, fib_0 + fib_1
    return fib_1

In [ ]:
# create an iterator
class FibIter:
    def __init__(self, n):
        self.n = n
        self.i = 0

    def __iter__(self):
        return self
    
    def __next__(self):
        if self.i >= self.n:
            raise StopIteration
        else:
            result = fib_standard(self.i)
            self.i += 1
            return result

In [81]:
timeit('list(FibIter(5000))', globals=globals(), number=1)

0.0044302999999672465

In [82]:
timeit('list(fib_gen(5000))', globals=globals(), number=1)  # the generator is 5x faster!

0.0009102999997594452

### Making an Iterable from a Generator

In [85]:
def squares_gen(n):
    for i in range(n):
        yield i  ** 2

sq = squares_gen(5)
list(sq), list(sq)  # the generator (i.e. an iterator) is exhausted after first list call

([0, 1, 4, 9, 16], [])

In [86]:
class Squares:
    def __init__(self, n):
        self.n = n

    def __iter__(self):
        return squares_gen(self.n)

In [88]:
sq = Squares(5)
list(sq), list(sq)  # now sq is an iterable, whose __iter__ method returns a new iterator each time

([0, 1, 4, 9, 16], [0, 1, 4, 9, 16])

In [89]:
# or we can move the squares_gen fn inside the Squares class
class Squares:
    def __init__(self, n):
        self.n = n

    def __iter__(self):
        return self.squares_gen(self.n)

    @staticmethod
    def squares_gen(n):
        for i in range(n):
            yield i  ** 2

In [90]:
sq = Squares(5)
list(sq), list(sq)

([0, 1, 4, 9, 16], [0, 1, 4, 9, 16])

### Card Deck example with generators

In [94]:
from collections import namedtuple

Card = namedtuple('Card', 'rank suit')
SUITS = ('Spades', 'Hearts', 'Diamonds', 'Clubs')
RANKS = tuple(range(2, 11)) + tuple('JQKA')


```
suit_ix = card_ix // len(RANKS)
rank_ix = card_ix % len(RANKS)
```

In [95]:
def card_gen():
    for i in range(len(SUITS) * len(RANKS)):
        suit = SUITS[i // len(RANKS)]
        rank = RANKS[i % len(RANKS)]
        card = Card(suit, rank)
        yield card

In [96]:
list(card_gen())

[Card(rank='Spades', suit=2),
 Card(rank='Spades', suit=3),
 Card(rank='Spades', suit=4),
 Card(rank='Spades', suit=5),
 Card(rank='Spades', suit=6),
 Card(rank='Spades', suit=7),
 Card(rank='Spades', suit=8),
 Card(rank='Spades', suit=9),
 Card(rank='Spades', suit=10),
 Card(rank='Spades', suit='J'),
 Card(rank='Spades', suit='Q'),
 Card(rank='Spades', suit='K'),
 Card(rank='Spades', suit='A'),
 Card(rank='Hearts', suit=2),
 Card(rank='Hearts', suit=3),
 Card(rank='Hearts', suit=4),
 Card(rank='Hearts', suit=5),
 Card(rank='Hearts', suit=6),
 Card(rank='Hearts', suit=7),
 Card(rank='Hearts', suit=8),
 Card(rank='Hearts', suit=9),
 Card(rank='Hearts', suit=10),
 Card(rank='Hearts', suit='J'),
 Card(rank='Hearts', suit='Q'),
 Card(rank='Hearts', suit='K'),
 Card(rank='Hearts', suit='A'),
 Card(rank='Diamonds', suit=2),
 Card(rank='Diamonds', suit=3),
 Card(rank='Diamonds', suit=4),
 Card(rank='Diamonds', suit=5),
 Card(rank='Diamonds', suit=6),
 Card(rank='Diamonds', suit=7),
 Card(rank

In [97]:
# rewrite with double loop to avoid fancy calculations
def card_gen():
    for suit in SUITS:
        for rank in RANKS:
            yield Card(rank, suit)

In [98]:
list(card_gen())

[Card(rank=2, suit='Spades'),
 Card(rank=3, suit='Spades'),
 Card(rank=4, suit='Spades'),
 Card(rank=5, suit='Spades'),
 Card(rank=6, suit='Spades'),
 Card(rank=7, suit='Spades'),
 Card(rank=8, suit='Spades'),
 Card(rank=9, suit='Spades'),
 Card(rank=10, suit='Spades'),
 Card(rank='J', suit='Spades'),
 Card(rank='Q', suit='Spades'),
 Card(rank='K', suit='Spades'),
 Card(rank='A', suit='Spades'),
 Card(rank=2, suit='Hearts'),
 Card(rank=3, suit='Hearts'),
 Card(rank=4, suit='Hearts'),
 Card(rank=5, suit='Hearts'),
 Card(rank=6, suit='Hearts'),
 Card(rank=7, suit='Hearts'),
 Card(rank=8, suit='Hearts'),
 Card(rank=9, suit='Hearts'),
 Card(rank=10, suit='Hearts'),
 Card(rank='J', suit='Hearts'),
 Card(rank='Q', suit='Hearts'),
 Card(rank='K', suit='Hearts'),
 Card(rank='A', suit='Hearts'),
 Card(rank=2, suit='Diamonds'),
 Card(rank=3, suit='Diamonds'),
 Card(rank=4, suit='Diamonds'),
 Card(rank=5, suit='Diamonds'),
 Card(rank=6, suit='Diamonds'),
 Card(rank=7, suit='Diamonds'),
 Card(rank

In [101]:
# create an iterable
class CardDeck:
    SUITS = ('Spades', 'Hearts', 'Diamonds', 'Clubs')
    RANKS = tuple(range(2, 11)) + tuple('JQKA')

    def __iter__(self):
        return CardDeck.card_gen()

    @staticmethod
    def card_gen():
        for suit in CardDeck.SUITS:
            for rank in CardDeck.RANKS:
                yield Card(rank, suit)

In [103]:
deck = CardDeck()
list(deck), list(deck)

([Card(rank=2, suit='Spades'),
  Card(rank=3, suit='Spades'),
  Card(rank=4, suit='Spades'),
  Card(rank=5, suit='Spades'),
  Card(rank=6, suit='Spades'),
  Card(rank=7, suit='Spades'),
  Card(rank=8, suit='Spades'),
  Card(rank=9, suit='Spades'),
  Card(rank=10, suit='Spades'),
  Card(rank='J', suit='Spades'),
  Card(rank='Q', suit='Spades'),
  Card(rank='K', suit='Spades'),
  Card(rank='A', suit='Spades'),
  Card(rank=2, suit='Hearts'),
  Card(rank=3, suit='Hearts'),
  Card(rank=4, suit='Hearts'),
  Card(rank=5, suit='Hearts'),
  Card(rank=6, suit='Hearts'),
  Card(rank=7, suit='Hearts'),
  Card(rank=8, suit='Hearts'),
  Card(rank=9, suit='Hearts'),
  Card(rank=10, suit='Hearts'),
  Card(rank='J', suit='Hearts'),
  Card(rank='Q', suit='Hearts'),
  Card(rank='K', suit='Hearts'),
  Card(rank='A', suit='Hearts'),
  Card(rank=2, suit='Diamonds'),
  Card(rank=3, suit='Diamonds'),
  Card(rank=4, suit='Diamonds'),
  Card(rank=5, suit='Diamonds'),
  Card(rank=6, suit='Diamonds'),
  Card(rank

In [107]:
# implement reversed method
class CardDeck:
    SUITS = ('Spades', 'Hearts', 'Diamonds', 'Clubs')
    RANKS = tuple(range(2, 11)) + tuple('JQKA')

    def __iter__(self):
        return CardDeck.card_gen()

    def __reversed__(self):
        return CardDeck.reversed_card_gen()

    @staticmethod
    def card_gen():
        for suit in CardDeck.SUITS:
            for rank in CardDeck.RANKS:
                yield Card(rank, suit)

    @staticmethod
    def reversed_card_gen():
        for suit in reversed(CardDeck.SUITS):
            for rank in reversed(CardDeck.RANKS):
                yield Card(rank, suit)

In [109]:
list(reversed(CardDeck()))  # properly reversed!

[Card(rank='A', suit='Clubs'),
 Card(rank='K', suit='Clubs'),
 Card(rank='Q', suit='Clubs'),
 Card(rank='J', suit='Clubs'),
 Card(rank=10, suit='Clubs'),
 Card(rank=9, suit='Clubs'),
 Card(rank=8, suit='Clubs'),
 Card(rank=7, suit='Clubs'),
 Card(rank=6, suit='Clubs'),
 Card(rank=5, suit='Clubs'),
 Card(rank=4, suit='Clubs'),
 Card(rank=3, suit='Clubs'),
 Card(rank=2, suit='Clubs'),
 Card(rank='A', suit='Diamonds'),
 Card(rank='K', suit='Diamonds'),
 Card(rank='Q', suit='Diamonds'),
 Card(rank='J', suit='Diamonds'),
 Card(rank=10, suit='Diamonds'),
 Card(rank=9, suit='Diamonds'),
 Card(rank=8, suit='Diamonds'),
 Card(rank=7, suit='Diamonds'),
 Card(rank=6, suit='Diamonds'),
 Card(rank=5, suit='Diamonds'),
 Card(rank=4, suit='Diamonds'),
 Card(rank=3, suit='Diamonds'),
 Card(rank=2, suit='Diamonds'),
 Card(rank='A', suit='Hearts'),
 Card(rank='K', suit='Hearts'),
 Card(rank='Q', suit='Hearts'),
 Card(rank='J', suit='Hearts'),
 Card(rank=10, suit='Hearts'),
 Card(rank=9, suit='Hearts'),


### Generator Expressions

In [113]:
g = (i ** 2 for i in range(5))
g, type(g)

(<generator object <genexpr> at 0x000002518BCE75F0>, generator)

In [114]:
[el for el in g], [el for el in g]  # exhausted on second call

([0, 1, 4, 9, 16], [])

In [122]:
import dis
exp = compile('[i**2 for i in range(5)]', filename='<string>', mode='eval')

In [123]:
dis.dis(exp)

  1           0 LOAD_CONST               0 (<code object <listcomp> at 0x000002518BF880E0, file "<string>", line 1>)
              2 LOAD_CONST               1 ('<listcomp>')
              4 MAKE_FUNCTION            0
              6 LOAD_NAME                0 (range)
              8 LOAD_CONST               2 (5)
             10 CALL_FUNCTION            1
             12 GET_ITER
             14 CALL_FUNCTION            1
             16 RETURN_VALUE

Disassembly of <code object <listcomp> at 0x000002518BF880E0, file "<string>", line 1>:
  1           0 BUILD_LIST               0
              2 LOAD_FAST                0 (.0)
        >>    4 FOR_ITER                12 (to 18)
              6 STORE_FAST               1 (i)
              8 LOAD_FAST                1 (i)
             10 LOAD_CONST               0 (2)
             12 BINARY_POWER
             14 LIST_APPEND              2
             16 JUMP_ABSOLUTE            4
        >>   18 RETURN_VALUE


In [125]:
gen = compile('(i**2 for i in range(5))', filename='<string>', mode='eval')
dis.dis(gen)  # same compilation as a list but with a generator instead

  1           0 LOAD_CONST               0 (<code object <genexpr> at 0x000002518BF88870, file "<string>", line 1>)
              2 LOAD_CONST               1 ('<genexpr>')
              4 MAKE_FUNCTION            0
              6 LOAD_NAME                0 (range)
              8 LOAD_CONST               2 (5)
             10 CALL_FUNCTION            1
             12 GET_ITER
             14 CALL_FUNCTION            1
             16 RETURN_VALUE

Disassembly of <code object <genexpr> at 0x000002518BF88870, file "<string>", line 1>:
  1           0 LOAD_FAST                0 (.0)
        >>    2 FOR_ITER                14 (to 18)
              4 STORE_FAST               1 (i)
              6 LOAD_FAST                1 (i)
              8 LOAD_CONST               0 (2)
             10 BINARY_POWER
             12 YIELD_VALUE
             14 POP_TOP
             16 JUMP_ABSOLUTE            2
        >>   18 LOAD_CONST               1 (None)
             20 RETURN_VALUE


### Yield From
Delegate yield to another iterator

In [126]:
def matrix(n):
    gen = ( (i * j for j in range(1, n+1))
        for i in range(1, n+1)
    )
    return gen

m = list(matrix(5))
m

[<generator object matrix.<locals>.<genexpr>.<genexpr> at 0x000002518BCE8890>,
 <generator object matrix.<locals>.<genexpr>.<genexpr> at 0x000002518BCE85F0>,
 <generator object matrix.<locals>.<genexpr>.<genexpr> at 0x000002518BCE8820>,
 <generator object matrix.<locals>.<genexpr>.<genexpr> at 0x000002518BCE8270>,
 <generator object matrix.<locals>.<genexpr>.<genexpr> at 0x000002518BCE8900>]

In [129]:
def matrix_iter(n):
    for row in matrix(n):
        for item in row:
            yield item  # yield items 1 by 1 from the row

In [130]:
[i for i in matrix_iter(3)]

[1, 2, 3, 2, 4, 6, 3, 6, 9]

In [131]:
def matrix_iter(n):
    for row in matrix(n):
        yield from row  # delegate iteration to the row iterator

In [132]:
[i for i in matrix_iter(3)]

[1, 2, 3, 2, 4, 6, 3, 6, 9]

In [136]:
file_1 = 'car-brands-1.txt'
file_2 = 'car-brands-2.txt'
file_3 = 'car-brands-3.txt'
files = file_1, file_2, file_3

In [138]:
# get a list of all car brands - brute force
brands = []
with open(file_1) as f:
    for brand in f:
        brands.append(brand.strip('\n'))

with open(file_2) as f:
    for brand in f:
        brands.append(brand.strip('\n'))

with open(file_3) as f:
    for brand in f:
        brands.append(brand.strip('\n'))

brands[:5]

['Alfa Romeo', 'Aston Martin', 'Audi', 'Bentley', 'Benz']

In [139]:
# iterative and generator approach
def brands(*files):
    for f_name in files:
        with open(f_name) as f:
            for line in f:
                yield line.strip('\n')

In [140]:
list(brands(*files))[:5]

['Alfa Romeo', 'Aston Martin', 'Audi', 'Bentley', 'Benz']

In [143]:
def brands(*files):
    for f_name in files:
        cleaned_data = gen_clean_data(f_name)
        for line in cleaned_data:
            yield line

def gen_clean_data(file):
    with open(file) as f:
        for row in f:
            yield(row.strip('\n'))

In [144]:
list(brands(*files))[:5]

['Alfa Romeo', 'Aston Martin', 'Audi', 'Bentley', 'Benz']

In [145]:
# consolidate with yield from
def brands(*files):
    for f_name in files:
        yield from gen_clean_data(f_name)

list(brands(*files))[:10]

['Alfa Romeo',
 'Aston Martin',
 'Audi',
 'Bentley',
 'Benz',
 'BMW',
 'Bugatti',
 'Cadillac',
 'Chevrolet',
 'Chrysler']